In [ ]:
import pandas as pd
import os
import wget
import PyPDF2
from PyPDF2 import PdfReader
import urllib.parse

In [ ]:
df = pd.read_csv("Studentenbewegung_pdfs.csv", encoding="utf-8")
df

In [ ]:
links = df["links"]
print(links)

In [ ]:
print(links[0])

In [ ]:
linklist = []

for element in links:
    related_links = element.split("; ")
    for link in related_links: 
        if link.startswith("https://d-nb.info/"):
            #print(link)
            linklist.append(link)

In [ ]:
print(len(linklist))

#### Alternative: 
```
linklist = []

for idn in df.idn: 
    link = "https://d-nb.info/" + idn + "/34"
    linklist.append(link) ```

## Herunterladen der PDF-Dateien

Gekürzte Liste erstellen (damit es schneller geht):

In [ ]:
testlist = linklist[60:70]
print(testlist)

Funktion zum Herunterladen definieren, die zunächst ein neues Verzeichnis für die gesammelten Dateien erstellt und diese dann herunterlädt und unter Nutzung der IDN als Dateiname abspeichert: 

In [ ]:
import requests

def download_text(testlist, save_directory):
    # Erstellen des später definierten Speicherverzeichnis, falls es dieses noch nicht gibt
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    for link in testlist: 
        # IDN ermitteln:
        idn = link.split('/')[-2]
        # Original-Dateiname und -Endung aus Content-Disposition header ermitteln und als Basis nutzen:
        response = requests.head(link)
        content_disposition = response.headers.get('Content-Disposition')
        orig_filename = content_disposition.split('filename=')[-1].strip('";')
        print(orig_filename)

        # Dateiname aus IDN und originalem Dateiname zusammensetzen: 
        file_name = f"{idn}_{orig_filename}"
        file_path = os.path.join(save_directory, file_name)
        print(f" Starte Download: {file_name}")
        try: 
            #Datei herunterladen:
            wget.download(link, out=file_path)
        except Exception as e:
            print(f"Fehler beim Herunterladen von {link}: {e}")

In [ ]:
save_directory = "downloads"

download_text(testlist, save_directory)

### Test: Einlesen einer Seite eines PDFs

In [ ]:
def test_read_pdf(file_path, file_name):
    reader = PdfReader(file_path + "/" + file_name)
    number_of_pages = len(reader.pages)
    page = reader.pages[10]
    text = page.extract_text()
    print(text)

In [ ]:
test_read_pdf("downloads", "1334711879.pdf")

### Einlesen einer kompletten PDF-Datei:

In [ ]:
def read_pdf_content(file_path, filename):
    try:
        reader = PdfReader(file_path + "/" + filename)
        number_of_pages = len(reader.pages)
        text = ""
        for page in reader.pages:
            text += page.extract_text()  # + "\n"
        return text
    except Exception as e:
        print(f"Fehler beim Einlesen der Datei {filename}: {e}")
        return "Fehler beim Einlesen der Datei"

In [ ]:
test = read_pdf_content("downloads", "1192129652_40257_1.pdf")
print(test)

### Einlesen **aller** Dateien aus dem "downloads"-Verzeichnis, die auf .pdf enden. Anschließend werden die Texte im Zusammenhang mit ihrem Dateinamen in ein Pandas_Dataframe überführt: 

In [ ]:
def get_text_from_pdfs(file_path):
    pdf_text = {}
    for filename in os.listdir(file_path):
        if filename.endswith(".pdf"):
            text = read_pdf_content(file_path, filename)
            pdf_text[filename] = text
    return pdf_text

In [ ]:
pdf_texts = get_text_from_pdfs("downloads")

In [ ]:
print(pdf_texts)

In [ ]:
#Löschen der ganzen Zeilenumbrüche ("\n"): 
pdf_texts = {key: value.replace('\n', ' ') for key, value in pdf_texts.items()}

In [ ]:
for key, value in pdf_texts.items():
    print(f"Key: {key}, Type of value: {type(value)}")
    if value is None:
        print(f"None value found for key: {key}")


In [ ]:
#Umwandeln in ein Dataframe:
df_texts = pd.DataFrame.from_dict(pdf_texts, orient='index', columns=['Text'])
df_texts.reset_index(inplace=True)
df_texts.columns = ['filename', 'text']
df_texts

In [ ]:
df_texts.to_hdf("pdf_texts.h5", key="df_texts")